In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [20]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [21]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [25]:
X = dataset.iloc[:, [2, 3]].values #age, estimated Salary
y = dataset.iloc[:, 4].values # purchased

# Splitting the dataset into the Training set and Test set


In [26]:
from sklearn.model_selection import train_test_split

In [29]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Feature Scaling

### Need to standardize the data. This elimates the 'oddballs.' Remove values that aren't centered around 0.

In [31]:
x_train[0:10]

array([[    44,  39000],
       [    32, 120000],
       [    38,  50000],
       [    32, 135000],
       [    52,  21000],
       [    53, 104000],
       [    39,  42000],
       [    38,  61000],
       [    36,  50000],
       [    36,  63000]])

In [32]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

/Users/cdavis/18Jan/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/cdavis/18Jan/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/cdavis/18Jan/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [33]:
x_train[0:10]

array([[ 0.58164944, -0.88670699],
       [-0.60673761,  1.46173768],
       [-0.01254409, -0.5677824 ],
       [-0.60673761,  1.89663484],
       [ 1.37390747, -1.40858358],
       [ 1.47293972,  0.99784738],
       [ 0.08648817, -0.79972756],
       [-0.01254409, -0.24885782],
       [-0.21060859, -0.5677824 ],
       [-0.21060859, -0.19087153]])

### Model Building and training
#### Need to convert training data into LightGBM dataset format (mandatory)

In [35]:
import lightgbm as lgb

/Users/cdavis/18Jan/venv/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [36]:
d_train = lgb.Dataset(x_train, label=y_train)

### python dictionary w/ various parameters

In [38]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt' # implementing gradient boosting
params['objective'] = 'binary' # this is classification
params['metric'] = 'binary_logloss' #classifiation
params['sub_feature'] = 0.5
params['num_leaves'] = 10 #only 10 since small
params['min_data'] = 50
params['max_depth'] = 10

In [39]:
clf = lgb.train(params, d_train, 100)

### PREDICTION

In [51]:
y_pred=clf.predict(x_test)

#### Output is a list of probabilities. Convert probabilities to binary prediction keeping threshold of 0.5

In [52]:
y_2 = y_pred

In [53]:
y_2

array([0.3274208 , 0.32358925, 0.32595692, 0.32595692, 0.31284985,
       0.32945164, 0.35115261, 0.42268495, 0.28817596, 0.39654625,
       0.31284985, 0.28817596, 0.31906633, 0.33692564, 0.3050809 ,
       0.41065339, 0.33692564, 0.3050809 , 0.50142195, 0.35115261,
       0.31284985, 0.50142195, 0.32358925, 0.50142195, 0.32945164,
       0.50142195, 0.30649366, 0.3050809 , 0.31906633, 0.32595692,
       0.30649366, 0.33692564, 0.39159395, 0.31906633, 0.28817596,
       0.28817596, 0.3050809 , 0.31284985, 0.35115261, 0.43855617,
       0.30649366, 0.31906633, 0.3050809 , 0.35115261, 0.42268495,
       0.28817596, 0.33692564, 0.41267428, 0.32945164, 0.4110622 ,
       0.50142195, 0.35115261, 0.30840859, 0.41065339, 0.41267428,
       0.43855617, 0.3050809 , 0.3050809 , 0.43855617, 0.32945164,
       0.35115261, 0.43855617, 0.3050809 , 0.42268495, 0.32945164,
       0.4110622 , 0.30649366, 0.28817596, 0.32358925, 0.33692564,
       0.41065339, 0.32595692, 0.28817596, 0.43855617, 0.30649

In [54]:
for i in range(0,99):
    if y_pred[i]>=.5:       # setting threshold to .5
        y_pred[i]=1
    else:  
        y_pred[i]=0

In [57]:
y_pred[99]=0

### RESULTS

##### Confusion matrix

In [58]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [59]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)

In [60]:
cm

array([[68,  0],
       [24,  8]])

In [61]:
accuracy

0.76